# Sequential Agent

This example shows how to create a simple sequential agent to perform a chain of subtasks by agents.

In [1]:
from typing import List

from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import BaseAgent
from aap_core.chain import BaseLLMChain
from aap_core.orchestration import SequentialAgent
from aap_core.types import AgentMessage
from aap_core.types import AgentResponse
from aap_dspy.chain import BaseSignatureAdapter, ChatCausalMultiTurnsChain

import dspy
from pydantic import Field

In [2]:
from typing import Dict


class QA(dspy.Signature):
    guide: str = dspy.InputField(optional=True)
    code: str = dspy.InputField(optional=True)
    doc: str = dspy.InputField(optional=True)
    review: str = dspy.InputField(optional=True)
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()
    history: dspy.History = dspy.InputField()


class SignatureAdapter(BaseSignatureAdapter[QA]):
    _signature_map: Dict[str, str]

    @classmethod
    def with_mapping(cls, prefill_dict: Dict[str, str], context_map: Dict[str, str]):
        obj = cls()
        obj._prefill_dict = prefill_dict
        for v in context_map.values():
            if v not in QA.fields.keys():
                raise ValueError(f"Invalid context key: {v}")
        obj._signature_map = context_map
        return obj

    def msg2sig(self, message: AgentMessage) -> List[QA]:
        history = dspy.History(messages=[])
        q, a = None, None
        for response in message.responses:
            if response[0] == "user":
                q = response[1]
            elif response[0] == "tool":
                pass
            elif response[0] == "system":
                pass
            else:
                a = response[1]

            if q is not None and a is not None:
                history.messages.append({"question": q, "answer": a})
                q, a = None, None

        sig_dict = {}
        for k, v in self._signature_map.items():
            if k.startswith("context.") and message.context is not None:
                sig_dict[v] = message.context.get(k.replace("context.", ""), "")
            elif k == "query":
                sig_dict[v] = message.query
        for k in QA.output_fields:
            sig_dict[k] = ""
        for k, v in QA.input_fields.items():
            if k not in sig_dict and v.annotation is not dspy.History:
                sig_dict[k] = ""
        sig_dict["guide"] = self._prefill_dict.get("guide", "")
        qa = QA(history=history, **sig_dict)
        return [qa]

    def sig2msg(self, signatures: List[QA], name: str) -> List[AgentResponse]:
        responses = []
        for signature in signatures:
            # responses.append(("user", signature.question))
            responses.append((name, signature.answer))
        return responses

## Define the agent

Create a simple agent for demonstration

In [3]:
class FunctionImplementationAgent(BaseAgent):
    chain: BaseLLMChain = Field(...)

    def execute(self, message: AgentMessage, **kwargs):
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        self.state = "idle"
        message.execution_result = "success"
        message.origin = self.card.name
        return message

In [4]:
model_code = dspy.LM(model="ollama_chat/granite3.3:8b", api_base="http://192.168.55.1:11434", api_key="", cache=False)
model_doc = dspy.LM(model="ollama_chat/mistral:7b-instruct-v0.3-q4_K_S", api_base="http://192.168.55.1:11434", api_key="", cache=False, max_tokens=32768)
model_review = dspy.LM(model="ollama_chat/qwen3:4b-thinking-2507-q4_K_M", api_base="http://192.168.55.1:11434", api_key="", cache=False, max_tokens=48000)
dspy.configure(adapter=dspy.ChatAdapter())

In [5]:
predictor = dspy.ChainOfThought(QA)

code_system_prompt = """You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Just the implementation, DO NOT comment anything else"""
code_adapter = SignatureAdapter.with_mapping(prefill_dict={"guide": code_system_prompt}, context_map={"query": "question"})
code_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=code_adapter).with_lm(model_code)
code_chain.final_response_as_context("code")

doc_system_prompt = """You are a great developer avocate.
You task is to write a document and in-code explain for the function.
Just the comment and explanation, DO NOT change the logic of the provided function.
The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code"""
doc_adapter = SignatureAdapter.with_mapping(prefill_dict={"guide": doc_system_prompt}, context_map={"query": "question", "context.code": "code"})
doc_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=doc_adapter).with_lm(model_doc)
doc_chain.final_response_as_context("doc")

review_system_prompt = """You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
review_adapter = SignatureAdapter.with_mapping(prefill_dict={"guide": review_system_prompt}, context_map={"query": "question", "context.doc": "doc"})
review_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=review_adapter).with_lm(model_review)

def state_callback(state: str):
    print(state)
code_skill = AgentSkill(
    id='code-skill',
    name="code skill",
    description="self-code skill",
    tags=['code']
)
code_card = AgentCard(
    name="code agent",
    description="self-code agent",
    skills=[code_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
doc_skill = AgentSkill(
    id='doc-skill',
    name="doc skill",
    description="self-doc skill",
    tags=['doc']
)
doc_card = AgentCard(
    name="doc agent",
    description="self-doc agent",
    skills=[doc_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
review_skill = AgentSkill(
    id='review-skill',
    name="review skill",
    description="self-review skill",
    tags=['review']
)
review_card = AgentCard(
    name="review agent",
    description="self-review agent",
    skills=[review_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
sequential_skill = AgentSkill(
    id='sequential-skill',
    name="sequential skill",
    description="self-sequential skill",
    tags=['sequential']
)
sequential_card = AgentCard(
    name="sequential agent",
    description="self-sequential agent",
    skills=[sequential_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
code_agent = FunctionImplementationAgent(chain=code_chain, card=code_card, state_change_callback=state_callback)
doc_agent = FunctionImplementationAgent(chain=doc_chain, card=doc_card, state_change_callback=state_callback)
review_agent = FunctionImplementationAgent(chain=review_chain, card=review_card, state_change_callback=state_callback)
agent = SequentialAgent(agents=[code_agent, doc_agent, review_agent], card=sequential_card, state_change_callback=state_callback)

In [6]:
query = "Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array."
message = agent.execute(AgentMessage(query=query))

for response in message.responses:
    name, msg = response
    print(name)
    print(msg)

sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:running)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:running)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:running))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:idle/((code agent:idle)-(doc agent:idle)-(review agent:idle))
review agent
```python
def find_biggest_fib(limit):
    """
    Returns the largest Fibonacci number less than or equal to the given limit.

    Args:
        limit (int): The upper bound for the Fibonacci number.

    Returns:
        int: The largest Fibonacci number <= limit.

    Example:
        >>> find_biggest_fib(10)
        8
 

In [7]:
dspy.inspect_history()





[2025-12-11T16:48:59.154103]

System message:

Your input fields are:
1. `guide` (str): 
2. `code` (str): 
3. `doc` (str): 
4. `review` (str): 
5. `question` (str): 
6. `history` (History):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## guide ## ]]
{guide}

[[ ## code ## ]]
{code}

[[ ## doc ## ]]
{doc}

[[ ## review ## ]]
{review}

[[ ## question ## ]]
{question}

[[ ## history ## ]]
{history}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `guide`, `code`, `doc`, `review`, `question`, `history`, produce the fields `answer`.


User message:

[[ ## guide ## ]]
You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the i